In [1]:
import sys
sys.path.append('../')
import networkx as nx
import time
from smodels.theory.exceptions import SModelSTheoryError as SModelSError
from smodels.tools.smodelsLogging import logger
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from smodels.theory.element import Element
from smodels.theory.topology import TopologyDict
from smodels.theory.crossSection import XSection,XSectionInfo,XSectionList
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.physicsUnits import fb, GeV
from smodels.theory.tree import Tree,ParticleNode
from smodels.theory.decomposer import cascadeDecay, addOneStepDecays
import itertools
%load_ext line_profiler

### Decomposer

In [2]:
def decompose(model, sigmacut= 0*fb, doCompress=True, doInvisible=True,
              minmassgap= 0*GeV):
    """
    Perform decomposition using the information stored in model.
    
    :param sigmacut: minimum sigma*BR to be generated, by default sigmacut = 0.1 fb
    :param doCompress: turn mass compression on/off
    :param doInvisible: turn invisible compression on/off
    :param minmassgap: maximum value (in GeV) for considering two R-odd particles
                       degenerate (only revelant for doCompress=True )
    :returns: list of topologies (TopologyList object)

    """
    t1 = time.time()
    
    xSectionList = model.xsections    
    pdgList = model.getValuesFor('pdg')

    if doCompress and minmassgap/GeV < 0.:
        logger.error("Asked for compression without specifying minmassgap. Please set minmassgap.")        
        raise SModelSError()

    if isinstance(sigmacut,(float,int)):
        sigmacut = float(sigmacut) * fb

    xSectionList.removeLowerOrder()
    # Order xsections by highest xsec value to improve performance
    xSectionList.sort()

    # Generate all primary nodes (e.g. PV > X+Y)
    # and assign the nodeWeight as the maximum cross-section
    productionTrees = []
    for pid in xSectionList.getPIDpairs():
        weight = xSectionList.getXsecsFor(pid)
        if weight < sigmacut:
            continue
        pv = ParticleNode(model.getParticlesWith(label='PV')[0],0,nodeWeight=weight)
        pv.xsection = xSectionList.getXsecsFor(pid)
        primaryMothers = [ParticleNode(model.getParticlesWith(pdg=pdg)[0],i+1) for i,pdg in enumerate(pid)]
        productionTrees.append(Tree({pv : primaryMothers}))

    # Sort production trees
    productionTrees = sorted(productionTrees, key = lambda t: t.getTreeWeight().getMaxXsec(), reverse=True)
    
    
    print('%i production trees' %len(productionTrees))
#     return productionTrees
    # For each production tree, produce all allowed cascade decays (above sigmacut):
    allTrees = []
    for tree in productionTrees:
#         print('len=',len(allTrees))
        allTrees += cascadeDecay(tree,sigmacut=sigmacut)

    print('%i decayed trees' %len(allTrees))
#     return allTrees

    # Create elements for each tree and combine equal elements
    smsTopDict = TopologyDict()

    allEls = []
    for tree in allTrees:
        newElement = Element(tree)
        newElement.weight = tree.getTreeWeight()
        
        
#         if newElement.tree.canonName == 11101010011011010000:
#             print('el created as:')
#             for d in newElement.tree.successors(newElement.tree.getTreeRoot()):
#                 print(d,d.canonName)
        
        
#         allEls.append(newElement)
        smsTopDict.addElement(newElement)                                                 
                                                    
#     smsTopDict.compressElements(doCompress, doInvisible, minmassgap)
#     smsTopDict._setElementIds()    
    print('total number of unique elements = %i' %len(smsTopDict.getElements()))
    print("decomposer done in %.2f s." % (time.time() -t1 ) )
    
#     return allEls
    return smsTopDict

### Load model

In [3]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
# slhafile = '../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


In [4]:
sigmacut = 10*fb
# sigmacut = 0.1*fb
topDict = decompose(model, sigmacut= sigmacut)
nTotal = 14647
nUnique = 5435 # uncompressed
print('(expected = %i trees)' %nTotal)
print('(expected = %i unique elements)' %nUnique)

22 production trees
219 decayed trees
total number of unique elements = 124
decomposer done in 1.20 s.
(expected = 14647 trees)
(expected = 5435 unique elements)


In [5]:
for c in sorted(topDict.keys()):
#     print(c,len(topDict[c]),topDict[c][0])
    print(c,len(topDict[c]),topDict[c][0].tree.treeToBrackets()[0])
    
    

110110101000 2 [[], [['q', 'q']]]
111010100110101000 61 [[['e-', 'nu']], [['nu', 'e+']]]
[] [N1/g/g, N3] []


SModelSTheoryError: 'Can not convert tree with Z2-violating decays to bracket: \n  (PV > gluino(1),gluino(2)), (gluino(1) > N1/b/b,b,b/b/g/g), (gluino(2) > N1/g/g,N3(3)), (N3(3) > N1,Z)'

In [ ]:
elList = topDict[11101010011011010000]
for el in elList:
#     if el.getCanonName() != 11101010011011010000:
#         continue
    el.sort()
    print('el == el',el == elList[0])
    print(el)
    for node in el.tree.nodes():
        print('  ',node,node.canonName)

In [ ]:
# %lprun -f addOneStepDecays decompose(model, sigmacut= sigmacut)

In [ ]:
ela = topDict[11101010011011010000][0]
elb = topDict[11101010011011010000][1]
# ela.sort()
# elb.sort()
print(ela)
print(elb)
print(ela == elb)

In [ ]:
for node in ela.tree.nodes():
    print(node,node.canonName)

In [ ]:
for node in elb.tree.nodes():
    print(node,node.canonName)

In [ ]:
elb.tree.sort()
for node in elb.tree.nodes():
    print(node,node.canonName)

In [ ]:
massA = list(zip(ela.tree.nodes(),ela.mass))
massB = list(zip(elb.tree.nodes(),elb.mass))

for im,m in enumerate(massA):
    print(m[0],massB[im][0])
    print('   mA=',m[1],'mB=',massB[im][1],'\n')

In [ ]:
ela.drawTree()

In [ ]:
elb.drawTree()